In [8]:
import os, random
import numpy as np
import pandas as pd
import datetime
from pprint import pprint
from pathlib import Path
import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero
#   https://docs.openmicroscopy.org/ome-model/5.6.3/developers/roi.html
#   https://github.com/HdBraanker/Spectral_Flow_Workflow/blob/main/script.Rmd
home = str(Path.home())

datestr = datetime.datetime.now().strftime('%Y_%m_%d')
print(datestr)

2024_10_08


In [9]:
### Subset Quant File from QuPath, which includes class annotations
clsPts = pd.read_csv(r'{}\Downloads\Slide-3015_potentialClasses.tsv'.format(home), sep="\t")  
clsPts

,CellTypes,x,y
0,<TBD>,3824.5,3397.1
1,<TBD>,3735.0,2470.5
2,<TBD>,3835.1,2950.5
3,<TBD>,3792.0,3347.9
4,<TBD>,3661.1,2464.0
...,...,...,...
1795,TumC,3389.2,2270.9
1796,TumC,3337.9,3448.3
1797,TumC,3255.8,5305.2
1798,TumC,3319.1,3547.5


In [10]:
# Settings for OMERO
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))
lab = envVars['labname']

conn = ezomero.connect(envVars['user'], envVars['pass'], lab, host=envVars['url'], port=4064, secure=True)
ezomero.print_projects(conn)

Projects:
	BMS_Teir2_2022:	102
	BMS_Tier1_2021:	201
	3D Analysis MxIF:	915
	IHC_v_MxIF_Developments:	306
	MelPanel4:	311
	BMS_CoreBiopsies_2023:	405
	IHC_v_MxIF_For_March2023_PathReview:	475
	Tassos_updated_2023:	457
	CosMx to CellDive Alignments:	606
	MISCELLANEOUS:	652
	IHC_v_MxIF_June6_PathReview:	712
	Elizabeth's Placenta Project:	751
	Lung Project :	752
	GM_CSF_Pilot:	807
	IHC_v_MxIF_Sept2023_PathReview:	713
	IHCs for CD36 and FASN:	865
	Mackenzie Study and RO1 Practice:	857
	20230620_IHC_MxIF_Dev_CD19_MPO_NRP2:	853
	BMS_Aim2_Pratice:	1005
	BMS_Aim2_Batch1:	1264
	BMS_Aim2_Batch1_Gastric:	1269
	BMS_2024_Mel_GI_H&E:	1274
	Pregnancy_Cancer:	1276


In [11]:
ezomero.print_datasets(conn, project=1264)
setDs = 2048

getImageByName = {}
imgs = ezomero.get_image_ids(conn, dataset=setDs)
for im in imgs:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imName = imgDesc.getName().replace(".ome.tiff","")
    getImageByName[imName] = im

Datasets in Project "BMS_Aim2_Batch1":
	BAD QUALITY:	2115
	Melanoma:	2048


In [12]:
def generate_rgba_values(length=3):
    rgba_list = []
    for _ in range(length):
        rgba = (
            random.randint(0, 255),  # Red
            random.randint(0, 255),  # Green
            random.randint(0, 255),  # Blue
            # round(random.uniform(0, 1), 2)  # Alpha (0.0 to 1.0)
            0.96
        )
        rgba_list.append(rgba)
    return rgba_list

In [16]:
im = getImageByName['SLIDE-3015_FullPanel']
clc = len(clsPts['CellTypes'].unique())
print("Have {} classes".format(clc))
myColors = generate_rgba_values(clc)
print(myColors)

## QuPath Exports in um, but OMERO uploads in px. Need pixel resolution.
resolution_um_per_pixel = 0.325   # Fusion  0.51

Have 6 classes
[(241, 241, 242, 0.96), (214, 32, 206, 0.96), (113, 181, 84, 0.96), (33, 161, 108, 0.96), (109, 78, 248, 0.96), (67, 27, 210, 0.96)]


In [17]:
i=0
for cl in clsPts['CellTypes'].unique():
    subTbl = clsPts[clsPts["CellTypes"] == cl]
    print("{} - {} rows".format(cl,subTbl.shape[0]))
    thesePoints = []
    for ele in subTbl.iterrows():
        row_dict = ele[1].to_dict()
        ptObj = ezomero.rois.Point( x=(row_dict['x']/resolution_um_per_pixel),  # Centroid X µm
                                    y=(row_dict['y']/resolution_um_per_pixel),  # Centroid Y µm
                                    label=row_dict['CellTypes']
                                  )
        thesePoints.append( ptObj )
    ezomero.post_roi(conn, im, thesePoints, stroke_color=myColors[i])
    i+=1


<TBD> - 300 rows
Epithelial - 300 rows
Neutro - 300 rows
SM - 300 rows
Treg - 300 rows
TumC - 300 rows
